In [ ]:
# Install necessary packages
%pip install simpy matplotlib seaborn plotly

# Import libraries
import simpy
import random
import matplotlib.pyplot as plt
from collections import deque
import numpy as np
import seaborn as sns
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Constants for the simulation
SIMULATION_TIME = 10000  # Total simulation time in minutes
VESSEL_INTER_ARRIVAL_TIME = 300  # Average time between vessel arrivals in minutes (5 hours)
CONTAINERS_PER_VESSEL = 150  # Number of containers per vessel
CRANE_MOVE_TIME = 3  # Time to move one container with a crane in minutes
TRUCK_ROUND_TRIP_TIME = 6  # Time for a truck to complete a round trip in minutes
NUM_BERTHS = 2  # Number of berths available
NUM_CRANES = 2  # Number of cranes available
NUM_TRUCKS = 3  # Number of trucks available

class Terminal:
    def __init__(self, env: simpy.Environment):
        """
        Initializes the terminal with its resources and trackers.
        """
        self.env = env
        self.berths = simpy.Resource(env, capacity=NUM_BERTHS)
        self.cranes = simpy.Resource(env, capacity=NUM_CRANES)
        self.trucks = simpy.Resource(env, capacity=NUM_TRUCKS)
        self.vessel_queue = []  # List to manage waiting vessels
        self.containers_moved = 0  # Counter for moved containers
        self.total_vessel_wait_time = 0  # Total wait time for vessels
        self.resource_manager = ResourceManager(env, self)
        self.environmental_tracker = EnvironmentalImpactTracker(self)
        self.predictive_scheduler = PredictiveScheduler()

    def vessel_arrival(self) -> None:
        """
        Process that simulates the arrival of vessels at the terminal.
        """
        while True:
            inter_arrival_time = self.predictive_scheduler.predict_next_arrival()
            yield self.env.timeout(inter_arrival_time)
            vessel_id = f"Vessel-{self.env.now}"
            print(f"{self.env.now}: {vessel_id} arrived")
            self.vessel_queue.append(vessel_id)
            self.env.process(self.handle_vessel(vessel_id))

    def handle_vessel(self, vessel_id: str) -> None:
        """
        Process for handling vessel berthing and unloading of containers.
        """
        arrival_time = self.env.now
        with self.berths.request() as berth_req:
            yield berth_req
            wait_time = self.env.now - arrival_time
            self.total_vessel_wait_time += wait_time
            self.vessel_queue.remove(vessel_id)
            print(f"{self.env.now}: {vessel_id} berthed after waiting {wait_time} minutes")

            with self.cranes.request() as crane_req, self.resource_manager.extra_cranes.request() as extra_crane_req:
                yield crane_req
                yield extra_crane_req
                for _ in range(CONTAINERS_PER_VESSEL):
                    with self.trucks.request() as truck_req, self.resource_manager.extra_trucks.request() as extra_truck_req:
                        yield truck_req
                        yield extra_truck_req
                        yield self.env.timeout(CRANE_MOVE_TIME)
                        self.containers_moved += 1
                        print(f"{self.env.now}: {vessel_id} - Container moved by crane")
                        self.env.process(self.truck_round_trip())
                        self.environmental_tracker.record_container_move()

            print(f"{self.env.now}: {vessel_id} departed")
            self.predictive_scheduler.update_historical_data(inter_arrival_time=wait_time)

    def truck_round_trip(self) -> None:
        """
        Process for simulating a truck's round trip.
        """
        yield self.env.timeout(TRUCK_ROUND_TRIP_TIME)
        self.environmental_tracker.record_truck_trip()

class ResourceManager:
    def __init__(self, env: simpy.Environment, terminal: Terminal):
        """
        Manages additional resources like extra cranes and trucks.
        """
        self.env = env
        self.terminal = terminal
        self.extra_cranes = simpy.Resource(env, capacity=1)
        self.extra_trucks = simpy.Resource(env, capacity=2)
        self.env.process(self.optimize_resources())

    def optimize_resources(self) -> None:
        """
        Periodically adjusts the allocation of extra resources based on queue lengths.
        """
        while True:
            yield self.env.timeout(60)  # Check every hour
            self.allocate_cranes()
            self.allocate_trucks()

    def allocate_cranes(self) -> None:
        """
        Dynamically allocates extra cranes based on the number of waiting vessels.
        """
        if len(self.terminal.vessel_queue) > 1 and self.extra_cranes.level == 0:
            yield self.extra_cranes.put(1)  # Add an extra crane
            print(f"{self.env.now}: Added an extra crane. Total cranes: {self.terminal.cranes.capacity + self.extra_cranes.level}")
        elif len(self.terminal.vessel_queue) == 0 and self.extra_cranes.level > 0:
            yield self.extra_cranes.get(1)  # Remove a crane
            print(f"{self.env.now}: Removed a crane. Total cranes: {self.terminal.cranes.capacity + self.extra_cranes.level}")

    def allocate_trucks(self) -> None:
        """
        Dynamically allocates extra trucks based on their utilization.
        """
        if self.terminal.trucks.count == self.terminal.trucks.capacity + self.extra_trucks.level and self.extra_trucks.level < 2:
            yield self.extra_trucks.put(1)  # Add an extra truck
            print(f"{self.env.now}: Added an extra truck. Total trucks: {self.terminal.trucks.capacity + self.extra_trucks.level}")
        elif self.terminal.trucks.count < (self.terminal.trucks.capacity + self.extra_trucks.level) // 2 and self.extra_trucks.level > 0:
            yield self.extra_trucks.get(1)  # Remove a truck
            print(f"{self.env.now}: Removed a truck. Total trucks: {self.terminal.trucks.capacity + self.extra_trucks.level}")

class EnvironmentalImpactTracker:
    def __init__(self, terminal: Terminal):
        """
        Tracks environmental impact such as emissions from crane moves and truck trips.
        """
        self.terminal = terminal
        self.total_emissions = 0
        self.crane_moves = 0
        self.truck_trips = 0

    def record_container_move(self) -> None:
        """
        Records emissions for each container move.
        """
        self.crane_moves += 1
        self.total_emissions += 0.5  # Assuming 0.5 units of emissions per crane move

    def record_truck_trip(self) -> None:
        """
        Records emissions for each truck trip.
        """
        self.truck_trips += 1
        self.total_emissions += 1  # Assuming 1 unit of emissions per truck trip

    def get_total_emissions(self) -> float:
        """
        Returns the total emissions recorded.
        """
        return self.total_emissions

class PredictiveScheduler:
    def __init__(self):
        """
        Predicts the next vessel arrival time based on historical data.
        """
        self.historical_data = deque(maxlen=100)
        self.base_arrival_time = VESSEL_INTER_ARRIVAL_TIME

    def predict_next_arrival(self) -> float:
        """
        Predicts the next vessel arrival time based on historical data.
        """
        if len(self.historical_data) < 10:
            return random.expovariate(1/self.base_arrival_time)
        return random.expovariate(1/np.mean(self.historical_data))

    def update_historical_data(self, inter_arrival_time: float) -> None:
        """
        Updates historical data with the latest inter-arrival time.
        """
        self.historical_data.append(inter_arrival_time)

class TerminalDashboard:
    def __init__(self, terminal: Terminal):
        """
        Initializes the dashboard to track and plot simulation data.
        """
        self.terminal = terminal
        self.vessel_queue_history = []  # History of vessel queue lengths
        self.containers_moved_history = []  # History of containers moved
        self.emissions_history = []  # History of total emissions
        self.resource_utilization_history = []  # History of resource utilization
        self.crane_moves_history = []  # History of crane moves
        self.truck_trips_history = []  # History of truck trips
        self.wait_times = []  # History of vessel wait times

    def update_data(self) -> None:
        """
        Updates the dashboard data with the latest simulation statistics.
        """
        self.vessel_queue_history.append(len(self.terminal.vessel_queue))
        self.containers_moved_history.append(self.terminal.containers_moved)
        self.emissions_history.append(self.terminal.environmental_tracker.get_total_emissions())
        berths_util = self.terminal.berths.count / self.terminal.berths.capacity
        cranes_util = self.terminal.cranes.count / (self.terminal.cranes.capacity + self.terminal.resource_manager.extra_cranes.capacity)
        trucks_util = self.terminal.trucks.count / (self.terminal.trucks.capacity + self.terminal.resource_manager.extra_trucks.capacity)
        self.resource_utilization_history.append((berths_util, cranes_util, trucks_util))
        self.crane_moves_history.append(self.terminal.environmental_tracker.crane_moves)
        self.truck_trips_history.append(self.terminal.environmental_tracker.truck_trips)
        self.wait_times.append(self.terminal.total_vessel_wait_time / (len(self.terminal.vessel_queue) + 1))

    def plot_results(self) -> None:
        """
        Plots the results of the simulation using Plotly for interactive visualizations.
        """
        # Define bright colors for the plots
        colors = {
            "vessel_queue": "cyan",
            "containers_moved": "lime",
            "emissions": "red",
            "berths_util": "magenta",
            "cranes_util": "yellow",
            "trucks_util": "orange",
            "wait_times": "blue",
            "crane_moves": "green",
            "truck_trips": "purple"
        }

        # Create a subplot layout
        fig = make_subplots(rows=3, cols=2, subplot_titles=(
            'Vessel Queue Over Time', 'Containers Moved Over Time', 'Total Emissions Over Time',
            'Resource Utilization Over Time', 'Distribution of Vessel Wait Times', 'Crane and Truck Activity Over Time'
        ))

        # Add traces to the subplot
        fig.add_trace(go.Scatter(y=self.vessel_queue_history, mode='lines', name='Vessel Queue Length', line=dict(color=colors["vessel_queue"])), row=1, col=1)
        fig.add_trace(go.Scatter(y=self.containers_moved_history, mode='lines', name='Containers Moved', line=dict(color=colors["containers_moved"])), row=1, col=2)
        fig.add_trace(go.Scatter(y=self.emissions_history, mode='lines', name='Total Emissions', line=dict(color=colors["emissions"])), row=2, col=1)

        berths_util, cranes_util, trucks_util = zip(*self.resource_utilization_history)
        fig.add_trace(go.Scatter(y=berths_util, mode='lines', name='Berth Utilization', line=dict(color=colors["berths_util"])), row=2, col=2)
        fig.add_trace(go.Scatter(y=cranes_util, mode='lines', name='Crane Utilization', line=dict(color=colors["cranes_util"])), row=2, col=2)
        fig.add_trace(go.Scatter(y=trucks_util, mode='lines', name='Truck Utilization', line=dict(color=colors["trucks_util"])), row=2, col=2)

        fig.add_trace(go.Histogram(x=self.wait_times, name='Vessel Wait Times', marker=dict(color=colors["wait_times"])), row=3, col=1)

        fig.add_trace(go.Scatter(y=self.crane_moves_history, mode='lines', name='Crane Moves', line=dict(color=colors["crane_moves"])), row=3, col=2)
        fig.add_trace(go.Scatter(y=self.truck_trips_history, mode='lines', name='Truck Trips', line=dict(color=colors["truck_trips"])), row=3, col=2)

        # Update layout settings for the plot
        fig.update_layout(title='Terminal Simulation Results', height=800, showlegend=False)
        fig.show()

# Create the SimPy environment
env = simpy.Environment()

# Create the terminal and dashboard
terminal = Terminal(env)
dashboard = TerminalDashboard(terminal)

# Start the vessel arrival process
env.process(terminal.vessel_arrival())

# Run the simulation for the specified duration
for _ in range(SIMULATION_TIME):
    env.step()  # Advance the simulation by one step
    dashboard.update_data()  # Update the dashboard with the latest data

# Plot the results after the simulation is complete
dashboard.plot_results()


209.5185669896801: Vessel-209.5185669896801 arrived
209.5185669896801: Vessel-209.5185669896801 berthed after waiting 0.0 minutes
212.5185669896801: Vessel-209.5185669896801 - Container moved by crane
215.5185669896801: Vessel-209.5185669896801 - Container moved by crane
218.5185669896801: Vessel-209.5185669896801 - Container moved by crane
221.5185669896801: Vessel-209.5185669896801 - Container moved by crane
224.5185669896801: Vessel-209.5185669896801 - Container moved by crane
227.5185669896801: Vessel-209.5185669896801 - Container moved by crane
230.5185669896801: Vessel-209.5185669896801 - Container moved by crane
233.5185669896801: Vessel-209.5185669896801 - Container moved by crane
236.5185669896801: Vessel-209.5185669896801 - Container moved by crane
239.5185669896801: Vessel-209.5185669896801 - Container moved by crane
242.5185669896801: Vessel-209.5185669896801 - Container moved by crane
245.5185669896801: Vessel-209.5185669896801 - Container moved by crane
248.5185669896801: